In [72]:
import os
from glob import glob
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm 

In [73]:
risultati = [y for x in os.walk("final_joined_result/") for y in glob(os.path.join(x[0], '*.csv'))]
risultati[:5]

['final_joined_result/triangle2_y.zip ExKMC.csv',
 'final_joined_result/churnONEHOT_y.zip-12_k skl_bis_kmeans.csv',
 'final_joined_result/compas-scores-two-yearsONEHOT_y.zip-2_True_k kmeans+dt.csv',
 'final_joined_result/s-set1_y.zip-4_True_k kmeans+dt.csv',
 'final_joined_result/wine.zip VarianceParTree.csv']

In [74]:
risultati_groupby_alg = dict()
risultati_groupby_dataset = dict()

for filename in risultati:
    alg = filename.split(" ")[-1][:-4]
    dataset = filename.split(" ")[0].split("\\")[-1][:-4]
    
    if alg not in risultati_groupby_alg:
        risultati_groupby_alg[alg] = []
    if dataset not in risultati_groupby_dataset:
        risultati_groupby_dataset[dataset] = []
    
    risultati_groupby_alg[alg].append(filename)
    risultati_groupby_dataset[dataset].append(filename)

In [75]:
res_values_uns = ["time", "silhouette", "calinski_harabasz", "davies_bouldin"]
res_values_sup = ["r_score", "adj_rand", "mut_info_score", "adj_mutual_info_score", "norm_mutual_info_score",
                                 "homog_score", "complete_score", "v_msr_score", "fwlks_mallows_score"]
res_values_all = res_values_uns + res_values_sup

In [76]:
df = pd.DataFrame()
lista = []

datasets = set()
algorithms = set()

for filename in risultati:
    alg = filename.split(" ")[-1][:-4]
    dataset = filename.split(" ")[0].split("/")[-1][:-4].replace(".zip", "").replace("-", "")
        
    
    datasets.add(dataset)
    algorithms.add(alg)
    
    lista.append(pd.read_csv(filename))
    
    lista[-1]["alg"] = alg
    if "wine" in dataset:
        lista[-1]["dataset"] = "wine"
    elif "home" in dataset:
        lista[-1]["dataset"] = "home"
    elif "ficoONEHOT" in dataset:
        lista[-1]["dataset"] = "ficoONEHOT"
    elif "fico" in dataset:
        lista[-1]["dataset"] = "fico"
    elif "bank" in dataset:
        lista[-1]["dataset"] = "bank"
    else:
        lista[-1]["dataset"] = dataset.split("_y")[0]+"_y"
    
df = pd.concat(lista)

df.columns

Index(['n_clusters', 'max_leaves', 'base_tree', 'max_nbr_values',
       'max_nbr_values_cat', 'time', 'silhouette', 'calinski_harabasz',
       'davies_bouldin', 'r_score', 'adj_rand', 'mut_info_score',
       'adj_mutual_info_score', 'norm_mutual_info_score', 'homog_score',
       'complete_score', 'v_msr_score', 'fwlks_mallows_score', 'alg',
       'dataset', 'init', 'n_init', 'max_iter', 'tol', 'verbose',
       'random_state', 'copy', 'algorithm', 'bisecting_strategy',
       'labels_as_tree_leaves', 'criterion', 'splitter', 'min_samples_split',
       'min_samples_leaf', 'min_weight_fraction_leaf', 'max_features',
       'max_leaf_nodes', 'min_impurity_decrease', 'class_weight', 'ccp_alpha',
       'max_depth', 'max_nbr_clusters', 'bic_eps', 'eps', 'min_samples',
       'metric', 'n_jobs', 'linkage', 'amount_initial_centers', 'kmax',
       'criteria_clf', 'criteria_reg', 'depth_factor', 'metric_con',
       'metric_cat', 'max_eps', 'n_components', 'oblique_splits',
       'max_o

In [77]:
df.dataset.unique()

array(['triangle2_y', 'churnONEHOT_y', 'compasscorestwoyearsONEHOT_y',
       'sset1_y', 'wine', 'churn_y', '2d10c_y', 'compasscorestwoyears_y',
       'diabetes_y', 'longsquare_y', 'iris_y', 'sset2_y', 'tetra_y',
       'aggregation_y', 'wdbc_y', 'adultONEHOT_y', 'zelnik5_y', '2dd31_y',
       '2d3cno123_y', 'curet24k_y', '2d4c_y', 'ionosphere_y',
       'german_creditONEHOT_y', 'curet12000n2D_y', 'home', 'triangle1_y',
       'adult_y', 'ficoONEHOT', 'german_credit_y', '2d4cno9_y',
       '2d20cno0_y', 'fico', 'vehicle_y', 'zelnik6_y', 'titanic_y',
       'curet02000n2D_y', 'bank'], dtype=object)

In [78]:
DS1 = ['datasets/syntetic/2d-3c-no123_y.zip', 'datasets/syntetic/s-set1_y.zip', 'datasets/syntetic/2d-20c-no0_y.zip', 'datasets/syntetic/triangle2_y.zip',
                  'datasets/syntetic/triangle1_y.zip', 'datasets/syntetic/cure-t0-2000n-2D_y.zip', 'datasets/syntetic/longsquare_y.zip', 'datasets/syntetic/aggregation_y.zip',
                  'datasets/syntetic/2d-4c-no9_y.zip', 'datasets/syntetic/2d-d31_y.zip', 'datasets/syntetic/cure-t2-4k_y.zip', 'datasets/syntetic/zelnik6_y.zip',
                  'datasets/syntetic/2d-10c_y.zip', 'datasets/syntetic/2d-4c_y.zip', 'datasets/syntetic/zelnik5_y.zip', 'datasets/syntetic/cure-t1-2000n-2D_y.zip',
                  'datasets/syntetic/s-set2_y.zip', 'datasets/syntetic/tetra_y.zip']

DS1 = list(map(lambda x: x.split("/")[-1][:-4].replace("-", ""), DS1))

DS1

['2d3cno123_y',
 'sset1_y',
 '2d20cno0_y',
 'triangle2_y',
 'triangle1_y',
 'curet02000n2D_y',
 'longsquare_y',
 'aggregation_y',
 '2d4cno9_y',
 '2dd31_y',
 'curet24k_y',
 'zelnik6_y',
 '2d10c_y',
 '2d4c_y',
 'zelnik5_y',
 'curet12000n2D_y',
 'sset2_y',
 'tetra_y']

In [79]:
DS2 = [x for x in df.dataset.unique() if "ONEHOT" in x] + ["compasscorestwoyears_y"] + ["iris_y", "wdbc_y"]

DS2

['churnONEHOT_y',
 'compasscorestwoyearsONEHOT_y',
 'adultONEHOT_y',
 'german_creditONEHOT_y',
 'ficoONEHOT',
 'compasscorestwoyears_y',
 'iris_y',
 'wdbc_y']

In [80]:
DS3 = [x for x in df.dataset.unique() if x not in DS1+DS2[:-2]]

DS3

['wine',
 'churn_y',
 'diabetes_y',
 'iris_y',
 'wdbc_y',
 'ionosphere_y',
 'home',
 'adult_y',
 'german_credit_y',
 'fico',
 'vehicle_y',
 'titanic_y',
 'bank']

In [81]:
DS4 = [x for x in df.dataset.unique() if x not in DS1+DS2[:-3]]

DS4

['wine',
 'churn_y',
 'compasscorestwoyears_y',
 'diabetes_y',
 'iris_y',
 'wdbc_y',
 'ionosphere_y',
 'home',
 'adult_y',
 'german_credit_y',
 'fico',
 'vehicle_y',
 'titanic_y',
 'bank']

In [84]:
DS_name = "DS4"

In [85]:
if DS_name == "DS1":
    DS = DS1
    max_nbr_values_list = [100, 1000, np.inf]
    max_nbr_values_cat_list = [20]
if DS_name == "DS2":
    DS = DS2
    max_nbr_values_list = [100, 1000, np.inf]
    max_nbr_values_cat_list = [20, 100]
if DS_name == "DS3":
    DS = DS3
    max_nbr_values_list = [20]
    max_nbr_values_cat_list = [20]
if DS_name == "DS4":
    DS = DS4
    max_nbr_values_list = [20, 100, 1000, np.inf]
    max_nbr_values_cat_list = [20, 100]

In [86]:
df[df.alg == "CLTree"].dataset.unique()

array(['diabetes_y', '2d10c_y', 'sset2_y', 'tetra_y', '2dd31_y',
       '2d4cno9_y', 'sset1_y', 'compasscorestwoyears_y', 'home',
       'aggregation_y', '2d4c_y', 'triangle1_y', 'ionosphere_y',
       'curet24k_y', 'zelnik5_y', 'german_credit_y', 'vehicle_y',
       'titanic_y', 'curet12000n2D_y', '2d3cno123_y', 'longsquare_y',
       'zelnik6_y', '2d20cno0_y', 'curet02000n2D_y', 'triangle2_y'],
      dtype=object)

In [87]:
df = df[df.dataset.isin(DS)].copy(deep=True)

df

,n_clusters,max_leaves,base_tree,max_nbr_values,max_nbr_values_cat,time,silhouette,calinski_harabasz,davies_bouldin,r_score,...,metric_cat,max_eps,n_components,oblique_splits,max_oblique_features,min_nr_instances,min_y,min_rd,threshold,branching_factor
0,NaN,NaN,NaN,20.0,20.0,0.935042,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,20.0,20.0,0.930818,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,20.0,20.0,0.932790,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,20.0,20.0,0.988993,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,20.0,20.0,0.969439,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,10.0,NaN,NaN,1000.0,100.0,0.107084,0.2732,174.0051,1.0712,0.5356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,10.0,NaN,NaN,1000.0,20.0,0.101034,0.2732,174.0051,1.0712,0.5356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,10.0,NaN,NaN,20.0,20.0,0.134736,0.2740,172.3922,1.0655,0.5324,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,10.0,NaN,NaN,inf,100.0,0.097218,0.2732,174.0051,1.0712,0.5356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
df.alg.unique()

array(['VarianceParTree', 'skl_DBSCAN', 'skl_bis_kmeans', 'pyc_xmeans',
       'skl_kmeans', 'ImpurityParTree2', 'kmeans+dt', 'ShallowTree',
       'skl_agglomerativeClust', 'PrincipalParTree', 'CLTree', 'kmodes',
       'CenterParTree2', 'skl_OPTICS', 'skl_birch', 'ExKMC'], dtype=object)

In [89]:
df = df[(df.alg == "CLTree")|(df.max_nbr_values.isin(max_nbr_values_list) & df.max_nbr_values_cat.isin(max_nbr_values_cat_list))].copy(deep=True)

df

,n_clusters,max_leaves,base_tree,max_nbr_values,max_nbr_values_cat,time,silhouette,calinski_harabasz,davies_bouldin,r_score,...,metric_cat,max_eps,n_components,oblique_splits,max_oblique_features,min_nr_instances,min_y,min_rd,threshold,branching_factor
0,NaN,NaN,NaN,20.0,20.0,0.935042,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,20.0,20.0,0.930818,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,20.0,20.0,0.932790,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,20.0,20.0,0.988993,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,20.0,20.0,0.969439,0.0977,211.8811,3.4489,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,10.0,NaN,NaN,1000.0,100.0,0.107084,0.2732,174.0051,1.0712,0.5356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,10.0,NaN,NaN,1000.0,20.0,0.101034,0.2732,174.0051,1.0712,0.5356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,10.0,NaN,NaN,20.0,20.0,0.134736,0.2740,172.3922,1.0655,0.5324,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,10.0,NaN,NaN,inf,100.0,0.097218,0.2732,174.0051,1.0712,0.5356,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
df.alg.unique()

array(['VarianceParTree', 'skl_DBSCAN', 'skl_bis_kmeans', 'pyc_xmeans',
       'ImpurityParTree2', 'skl_kmeans', 'kmeans+dt', 'ShallowTree',
       'skl_agglomerativeClust', 'PrincipalParTree', 'CLTree', 'kmodes',
       'CenterParTree2', 'skl_OPTICS', 'skl_birch', 'ExKMC'], dtype=object)

In [91]:
order = "silhouette"
#order = "norm_mutual_info_score"

metriche = dict([#Ascending?
    ('silhouette', False), #max
    ('time', True), #min
    ('calinski_harabasz', False),
    ('davies_bouldin', True),
    ('r_score', False),
    ('adj_rand', False),
    ('mut_info_score', False),
    ('adj_mutual_info_score', False),
    ('norm_mutual_info_score', False),
    ('homog_score', True),
    ('complete_score', False),
    ('v_msr_score', False),
    ('fwlks_mallows_score', False)
])

In [92]:
df_subset = df.sort_values(['dataset', 'alg', order, 'time'], ascending=[False, False, metriche[order], True]).groupby(['dataset', 'alg']).nth(0).reset_index()

df_subset[res_values_sup+["time"]]

,r_score,adj_rand,mut_info_score,adj_mutual_info_score,norm_mutual_info_score,homog_score,complete_score,v_msr_score,fwlks_mallows_score,time
0,0.5946,0.1813,0.0809,0.1318,0.1318,0.1465,0.1198,0.1318,0.6515,16.434910
1,0.5725,-0.0065,0.0001,0.0001,0.0002,0.0001,0.0002,0.0002,0.7014,1.713524
2,0.5946,0.1813,0.0809,0.1318,0.1318,0.1465,0.1198,0.1318,0.6515,1.758047
3,0.5883,-0.0147,0.0005,0.0011,0.0011,0.0008,0.0017,0.0011,0.7307,19.332500
4,0.6064,0.0016,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.7533,0.891088
...,...,...,...,...,...,...,...,...,...,...
191,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.878600
192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.430120
193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.931088
194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.067898


In [93]:
df_subset.to_csv(f"{DS_name} order by {order}.csv")

In [94]:
#df_subset["rank_per_dataset"] = df_subset.groupby("dataset")[order].rank(ascending=metriche[order]).astype(int)

#df_subset.sort_values(["dataset", "rank_per_dataset"])[["rank_per_dataset", order, "alg", "dataset"]]

In [95]:
df_agg = df_subset.groupby("alg")[res_values_all].agg(["mean", "std", "min", "max"])

df_agg.head()

time                                     silhouette  \
                        mean          std       min          max       mean   
alg                                                                           
CLTree            434.054679  1060.476128  7.012719  2837.967403  -0.022986   
CenterParTree2     22.675457    74.505513  0.073068   270.214524   0.121615   
ExKMC               0.026708     0.024370  0.002252     0.071636   0.417600   
ImpurityParTree2    1.055368     1.518923  0.042267     5.145525   0.167671   
PrincipalParTree    0.878254     1.483417  0.010190     5.620932   0.284414   

                                           calinski_harabasz               \
                       std     min     max              mean          std   
alg                                                                         
CLTree            0.050620 -0.1145  0.0235          1.819443     1.074661   
CenterParTree2    0.178956 -0.0833  0.6575       1016.744292  2183.338906   
ExKMC             0.210655  0.1745  0.8047        582.962200   657.629763   
ImpurityParTree2  0.182730 -0.0047  0.6613        791.930764  2060.934987   
PrincipalParTree  0.207410  0.0094  0.6613       1962.952236  4503.834509   

                  ... complete_score         v_msr_score                    \
                  ...            min     max        mean       std     min   
alg               ...                                                        
CLTree            ...         0.0034  0.3518    0.106186  0.091483  0.0034   
CenterParTree2    ...         0.0076  0.4993    0.131736  0.145945  0.0064   
ExKMC             ...         0.0389  1.0000    0.262043  0.283439  0.0243   
ImpurityParTree2  ...         0.0002  0.5382    0.109136  0.151590  0.0002   
PrincipalParTree  ...         0.0003  0.8835    0.247736  0.296271  0.0004   

                         fwlks_mallows_score                            
                     max                mean       std     min     max  
alg                                                                     
CLTree            0.2392            0.594186  0.130906  0.3316  0.7700  
CenterParTree2    0.4390            0.588973  0.074260  0.4245  0.6860  
ExKMC             0.7337            0.633300  0.158392  0.4278  0.8523  
ImpurityParTree2  0.4110            0.656045  0.105396  0.4494  0.7852  
PrincipalParTree  0.8324            0.658109  0.143312  0.4308  0.9412  

[5 rows x 52 columns]

In [96]:
def getWin(metrica):
    ascending = metriche[metrica]
    df_tmp = df_subset[["alg", "dataset", metrica]].copy()
    df_tmp["rank"] = df_subset.groupby("dataset")[metrica].rank(ascending=ascending, method="min", na_option="top" if ascending else "bottom").astype(int)
    return df_tmp.groupby("alg")[["rank"]].agg(c=("rank", len), win=("rank", lambda x: len(x[x == 1])), rank_mean=("rank", np.mean))

In [97]:
df_agg = df_agg[df_agg.index != "pyc_agglomerativeClust"].copy(deep=True)

for metrica in res_values_all:
    df_tmp = getWin(metrica)
    df_agg[(metrica, "win")] = df_tmp.win
    df_agg[(metrica, "count")] = df_tmp.c
    df_agg[(metrica, "rank_mean")] = df_tmp.rank_mean

df_agg[df_agg.columns.sort_values()]

adj_mutual_info_score                            \
                                       count     max      mean     min   
alg                                                                      
CLTree                                     7  0.2375  0.103300  0.0002   
CenterParTree2                            13  0.4310  0.130073  0.0053   
ExKMC                                      8  0.7316  0.259843  0.0231   
ImpurityParTree2                          14  0.4102  0.107664  0.0001   
PrincipalParTree                          14  0.8324  0.246527  0.0001   
ShallowTree                                8  0.7316  0.235857 -0.0011   
VarianceParTree                           14  0.5204  0.127845 -0.0003   
kmeans+dt                                 14  0.7316  0.134218 -0.0019   
kmodes                                    13  0.3405  0.087927 -0.0004   
pyc_xmeans                                13  0.7316  0.162827 -0.0003   
skl_DBSCAN                                13  0.6900  0.085873  0.0000   
skl_OPTICS                                13  0.7316  0.142055 -0.0003   
skl_agglomerativeClust                    13  0.7316  0.070727 -0.0003   
skl_birch                                 13  0.6900  0.155282 -0.0007   
skl_bis_kmeans                            13  0.7316  0.177918 -0.0009   
skl_kmeans                                13  0.7316  0.182018 -0.0009   

                                               adj_rand                    \
                       rank_mean       std win    count     max      mean   
alg                                                                         
CLTree                  6.000000  0.091030   2        7  0.1875  0.081429   
CenterParTree2          6.538462  0.144350   5       13  0.3251  0.101064   
ExKMC                   3.375000  0.283559   2        8  0.6766  0.253000   
ImpurityParTree2        7.071429  0.150754   3       14  0.5335  0.095682   
PrincipalParTree        3.857143  0.296145   5       14  0.8817  0.256791   
ShallowTree             5.875000  0.281311   2        8  0.6361  0.233800   
VarianceParTree         6.214286  0.188058   4       14  0.5927  0.142200   
kmeans+dt               6.857143  0.237981   4       14  0.5681  0.136236   
kmodes                  7.307692  0.104356   2       13  0.3886  0.085555   
pyc_xmeans              6.461538  0.259225   4       13  0.6765  0.154600   
skl_DBSCAN              8.846154  0.203233   2       13  0.5438  0.063509   
skl_OPTICS              5.615385  0.213613   4       13  0.5681  0.084273   
skl_agglomerativeClust  8.846154  0.219228   3       13  0.5681  0.056636   
skl_birch               6.923077  0.235423   3       13  0.5750  0.153100   
skl_bis_kmeans          5.153846  0.250753   3       13  0.6707  0.171945   
skl_kmeans              4.846154  0.249042   3       13  0.6707  0.174955   

                        ...       time                  v_msr_score          \
                        ...  rank_mean          std win       count     max   
alg                     ...                                                   
CLTree                  ...  15.428571  1060.476128   0           7  0.2392   
CenterParTree2          ...  11.230769    74.505513   0          13  0.4390   
ExKMC                   ...   3.375000     0.024370   2           8  0.7337   
ImpurityParTree2        ...   8.000000     1.518923   0          14  0.4110   
PrincipalParTree        ...   6.714286     1.483417   0          14  0.8324   
ShallowTree             ...   7.375000     0.288354   0           8  0.7337   
VarianceParTree         ...  11.142857     5.224982   0          14  0.5240   
kmeans+dt               ...   7.000000     0.389132   1          14  0.7337   
kmodes                  ...   9.461538     2.413834   0          13  0.3406   
pyc_xmeans              ...   2.692308     0.111967   4          13  0.7337   
skl_DBSCAN              ...   4.230769     5.406173   1          13  0.6925   
skl_OPTICS              ...  13.923077   111.8

In [98]:
lista = [ 
    'CenterParTree2', 
    'ImpurityParTree2', 
    'PrincipalParTree',
    'VarianceParTree', 
    'CLTree',
    'kmeans+dt', 
    'ExKMC',
    'ShallowTree',
    
    'skl_kmeans',
    'skl_bis_kmeans',
    'pyc_xmeans',
    'kmodes', 
    'skl_birch', 
    'skl_agglomerativeClust', 
    'skl_DBSCAN',
    'skl_OPTICS', 
    'skl_agglomerativeClust',     
]
df_agg["indice"] = list(map(lambda x: lista.index(x), df_agg.index.tolist()))
df_agg[df_agg.columns.sort_values()].sort_values(by="indice").drop(columns="indice")\
        .to_excel(f"{DS_name} order by {order}.xlsx")

/tmp/ipykernel_23124/645413274.py:22: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_agg[df_agg.columns.sort_values()].sort_values(by="indice").drop(columns="indice")\


In [99]:
df.alg.unique()

array(['VarianceParTree', 'skl_DBSCAN', 'skl_bis_kmeans', 'pyc_xmeans',
       'ImpurityParTree2', 'skl_kmeans', 'kmeans+dt', 'ShallowTree',
       'skl_agglomerativeClust', 'PrincipalParTree', 'CLTree', 'kmodes',
       'CenterParTree2', 'skl_OPTICS', 'skl_birch', 'ExKMC'], dtype=object)